In [10]:
import pandas as pd
from elasticsearch import Elasticsearch, helpers
import logging

In [11]:

logging.basicConfig(level=logging.INFO)

es = Elasticsearch(
    "http://192.168.1.230:30920",
    verify_certs=False,
    request_timeout=60,
    retry_on_timeout=True
)

index_name = "noms_prenoms"

df = pd.read_parquet("nom_prenoms_overlay_vf_lower_trim.parquet")

df = df.fillna(0)

# On fait une séparation des données pour constituer un sous objet pour l'année
def generate_docs(df):
    for i, row in df.iterrows():
        row_dict = row.to_dict()
        freq = {k: v for k, v in row_dict.items() if k.startswith("annee_")}
        meta = {k: v for k, v in row_dict.items() if not k.startswith("annee_")}
        yield {
            "_index": index_name,
            "_id": i,
            "_source": {
                **meta,
                "frequences": freq
            }
        }

if not es.indices.exists(index=index_name):
    # On construit dynamiquement le mapping pour le champ 'frequences'
    freq_mapping = {
        f"annee_{year}": {"type": "float"}
        for year in range(1886, 2021)
    }

    es.indices.create(
        index=index_name,
        body={
            "settings": {
                "number_of_shards": 4,
                "number_of_replicas": 1
            },
            "mappings": {
                "properties": {
                    "nom": {"type": "keyword"},
                    "partie_prenom": {"type": "integer"},
                    "nom_famille": {"type": "integer"},
                    "prenom_unique": {"type": "integer"},
                    "nombre_homme_prenom": {"type": "integer"},
                    "nombre_homme_nom_famille": {"type": "integer"},
                    "nombre_femme_prenom": {"type": "integer"},
                    "nombre_femme_nom_famille": {"type": "integer"},
                    "frequences": {
                        "type": "object",
                        "properties": freq_mapping
                    }
                }
            }
        }
    )


INFO:elastic_transport.transport:HEAD http://192.168.1.230:30920/noms_prenoms [status:200 duration:0.025s]


In [ ]:

# Insertion bulk avec gestion d'erreur pour les données qui pourraient manquées dans la base
try:
    success, errors = helpers.bulk(
        es,
        generate_docs(df),
        stats_only=False,
        raise_on_error=False
    )

    print(f"{success} documents insérés avec succès dans '{index_name}'")

    if errors:
        print(f"{len(errors)} document(s) ont échoué à l'insertion.")
        for err in errors[:5]:
            print(err)

except Exception as e:
    print("Erreur globale lors de l'insertion bulk :", e)


INFO:elastic_transport.transport:PUT http://192.168.1.230:30920/_bulk [status:200 duration:0.241s]
INFO:elastic_transport.transport:PUT http://192.168.1.230:30920/_bulk [status:200 duration:0.202s]
INFO:elastic_transport.transport:PUT http://192.168.1.230:30920/_bulk [status:200 duration:0.240s]
INFO:elastic_transport.transport:PUT http://192.168.1.230:30920/_bulk [status:200 duration:0.226s]
INFO:elastic_transport.transport:PUT http://192.168.1.230:30920/_bulk [status:200 duration:0.231s]
INFO:elastic_transport.transport:PUT http://192.168.1.230:30920/_bulk [status:200 duration:0.207s]
INFO:elastic_transport.transport:PUT http://192.168.1.230:30920/_bulk [status:200 duration:0.217s]
INFO:elastic_transport.transport:PUT http://192.168.1.230:30920/_bulk [status:200 duration:0.221s]
INFO:elastic_transport.transport:PUT http://192.168.1.230:30920/_bulk [status:200 duration:0.200s]
INFO:elastic_transport.transport:PUT http://192.168.1.230:30920/_bulk [status:200 duration:0.202s]
INFO:elast

1868968 documents insérés avec succès dans 'noms_prenoms'
